# Part I: Data loading and processing

In [100]:
#Import necessary libraries

import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import json

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!python -m pip install folium
import folium

print('Libraries imported')


Libraries imported


In [101]:
# Load data fron URL

d = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [102]:
len(d)

3

In [113]:
# Transform data into dataframe

df = pd.DataFrame(d[0])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [114]:
# Size before processing

df.shape

(180, 3)

In [115]:
#Drop the "Not assigned" borough

df = df[df.Borough != "Not assigned"].reset_index(drop=True)

print("Size after dropping 'Not assigned' borough: ", df.shape)
df.head()


Size after dropping 'Not assigned' borough:  (103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [116]:
# Group by Postal Code. More than one neighborhood can exist in one postal code area.

df = df.groupby(['Postal Code'], as_index=False).agg(lambda x: ','.join(x))
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [117]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

df['Neighbourhood'].replace("Not assigned", df["Borough"],inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [118]:
df.shape

(103, 3)

*** <font size = "4"> The shape of data frame after cleaning consists of 103 rows and 3 columns. </font> 

#  Part II: Latitude and longitude coordinates of each neighborhood

In [119]:
#Loading geospatial data from url

geo_coor = pd.read_csv("http://cocl.us/Geospatial_data")

In [120]:
# Transform data into dataframe

geo_df = pd.DataFrame(geo_coor)
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [121]:
# Merge both dataframe by adding on Latitude and Longitude

df = df.merge(geo_df, on= "Postal Code")

df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<font size="4">Latitue and Longitude coordinates successfully added!</font>

# Part III Explore and clustering the neighborhoods

In [122]:
#Extract data of neighbourhood in Downtown Toronto into dataframe 

toronto_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [123]:
#Search for the geographical coordinate

address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [124]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Create a function to repeat the process to all neighbourhood in Downtown Toronto**

In [125]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [126]:
CLIENT_ID ='*****'
CLIENT_SECRET='*****'
VERSION = '20180604'
LIMIT = 50

toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [127]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 179 uniques categories.


**Analyze each neighborhood**

In [128]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [129]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.02,0.00,0.00,0.0,0.000000,0.0,0.000000,0.04,0.0,0.0000,0.0,0.02,0.02,0.04,0.0,0.0,0.02,0.0000,0.00,0.0,0.02,0.0,0.00,0.00,0.0,0.0,0.040000,0.0,0.000000,0.0,0.04,0.0,0.0,0.02,0.04,0.080000,0.0,0.0,0.0,0.0,0.0,0.02,0.00,0.02,0.0,0.0,0.02,0.00,0.0,0.02,0.00,0.000000,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.04,0.0,0.0,0.02,0.0,0.0,0.02,0.02,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.02,0.02,0.000000,0.02,0.00,0.0000,0.0,0.0,0.00,0.0,0.02,0.0,0.00,0.00,0.020000,0.02,0.02,0.0,0.00,0.0,0.02,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.02,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.020000,0.0,0.00,0.0,0.02,0.00,0.0000,0.0,0.0,0.00,0.00,0.02,0.00,0.0,0.0000,0.040000,0.0,0.00,0.00,0.00,0.00,0.0,0.0000,0.04,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.02,0.0,0.02,0.02,0.0,0.0,0.0,0.02,0.00,0.00,0.0,0.0,0.02,0.0,0.0,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0625,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0625,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.00,0.062500,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.00,0.0625,0.0,0.0,0.00,0.0,0.00,0.0,0.00,

**New size after grouping**

In [130]:
toronto_grouped.shape

(19, 179)

**Print out top 3 venues in each neighborhood**

In [131]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1  Seafood Restaurant  0.04
2                Café  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1      Sandwich Place  0.06
2  Italian Restaurant  0.04


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12


----Church and Wellesley----
              venue  freq
0  Sushi Restaurant  0.06
1           Gay Bar  0.06
2        Restaurant  0.06


----Commerce Court, Victoria Hotel----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.10
2        Hotel  0.08


----First Canadian Place, Underground city----
         venue  freq
0         Café  0.12
1  Coffee Shop  0.10
2   Restaurant  0.06




**Cluster the neighborhood**

In [151]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

kmeans.labels_

array([1, 3, 4, 2, 1, 4, 4, 1, 1, 1, 4, 4, 1, 0, 1, 1, 1, 4, 1],
      dtype=int32)

In [146]:
toronto_grouped_clustering 

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.000000,0.00,0.02,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.040000,0.000000,0.000000,0.00,0.02,0.02,0.040000,0.000000,0.00,0.02,0.0000,0.000000,0.00,0.020000,0.00,0.00,0.00,0.00,0.000000,0.040000,0.00,0.000000,0.000000,0.04,0.000000,0.000000,0.02,0.04,0.080000,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.00,0.02,0.000000,0.000000,0.020000,0.00,0.000000,0.02,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.040000,0.00,0.00,0.02,0.00,0.00,0.02,0.020000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.02,0.02,0.000000,0.020000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.020000,0.020000,0.02,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.020000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.020000,0.00,0.00,0.0000,0.040000,0.00,0.00,0.00,0.00,0.000000,0.00,0.0000,0.04,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.02,0.00,0.020000,0.02,0.000000,0.00,0.00,0.020000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00
1,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.062500,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.0625,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0

**Show the 10 most common venues in each neighbourhood**

In [147]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [148]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Farmers Market,Seafood Restaurant,Bakery,Café,Cheese Shop,Cocktail Bar,Beer Bar,Bistro
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop,Bar,Sculpture Garden
2,Central Bay Street,Coffee Shop,Sandwich Place,Bubble Tea Shop,Burger Joint,Café,Italian Restaurant,Wine Bar,Middle Eastern Restaurant,Ice Cream Shop,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Diner,Nightclub
4,Church and Wellesley,Restaurant,Gay Bar,Coffee Shop,Sushi Restaurant,Yoga Studio,Men's Store,Japanese Restaurant,Ramen Restaurant,Bookstore,Breakfast Spot


In [149]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Trail,Playground,Wine Bar,Concert Hall,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Café,Coffee Shop,Bakery,Convenience Store,Pub,Pizza Place,Italian Restaurant,Restaurant,Beer Store,Diner
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Restaurant,Gay Bar,Coffee Shop,Sushi Restaurant,Yoga Studio,Men's Store,Japanese Restaurant,Ramen Restaurant,Bookstore,Breakfast Spot
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Coffee Shop,Bakery,Pub,Café,Park,Restaurant,Breakfast Spot,Theater,Yoga Studio,Cosmetics Shop
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Café,Theater,Clothing Store,Italian Restaurant,Bookstore,Ramen Restaurant,Middle Eastern Restaurant,Thai Restaurant,Sandwich Place


In [150]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Most neighbourhoods are in Cluster 2 and Cluster 5.** 
**So we will examine both clusters for verification.**

In [155]:
#Cluster 2

cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Café,Coffee Shop,Bakery,Convenience Store,Pub,Pizza Place,Italian Restaurant,Restaurant,Beer Store,Diner
2,Downtown Toronto,1,Restaurant,Gay Bar,Coffee Shop,Sushi Restaurant,Yoga Studio,Men's Store,Japanese Restaurant,Ramen Restaurant,Bookstore,Breakfast Spot
4,Downtown Toronto,1,Coffee Shop,Café,Theater,Clothing Store,Italian Restaurant,Bookstore,Ramen Restaurant,Middle Eastern Restaurant,Thai Restaurant,Sandwich Place
5,Downtown Toronto,1,Café,Coffee Shop,Restaurant,Cosmetics Shop,Park,Creperie,Hotel,Farmers Market,Seafood Restaurant,Gastropub
6,Downtown Toronto,1,Coffee Shop,Restaurant,Farmers Market,Seafood Restaurant,Bakery,Café,Cheese Shop,Cocktail Bar,Beer Bar,Bistro
8,Downtown Toronto,1,Café,Coffee Shop,Steakhouse,American Restaurant,Concert Hall,Hotel,Restaurant,Bakery,Seafood Restaurant,Bar
9,Downtown Toronto,1,Coffee Shop,Aquarium,Park,Brewery,Plaza,Hotel,Café,Theater,Basketball Stadium,Roof Deck
12,Downtown Toronto,1,Café,Sandwich Place,Bookstore,Bar,Japanese Restaurant,Restaurant,Bakery,Italian Restaurant,Beer Bar,Beer Store
13,Downtown Toronto,1,Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Dessert Shop,Burger Joint,Vietnamese Restaurant,Caribbean Restaurant,Pizza Place,Bakery
15,Downtown Toronto,1,Café,Restaurant,Beer Bar,Creperie,Italian Restaurant,Japanese Restaurant,Farmers Market,Pub,Seafood Restaurant,Cheese Shop


In [156]:
#Cluster 5

cluster_5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,4,Coffee Shop,Bakery,Pub,Café,Park,Restaurant,Breakfast Spot,Theater,Yoga Studio,Cosmetics Shop
7,Downtown Toronto,4,Coffee Shop,Sandwich Place,Bubble Tea Shop,Burger Joint,Café,Italian Restaurant,Wine Bar,Middle Eastern Restaurant,Ice Cream Shop,Japanese Restaurant
10,Downtown Toronto,4,Coffee Shop,Café,Seafood Restaurant,Restaurant,Hotel,Japanese Restaurant,Beer Bar,French Restaurant,Bakery,Bar
11,Downtown Toronto,4,Café,Coffee Shop,Restaurant,Hotel,Gym,Deli / Bodega,American Restaurant,Beer Bar,Japanese Restaurant,Seafood Restaurant
16,Downtown Toronto,4,Café,Coffee Shop,Hotel,Restaurant,American Restaurant,Deli / Bodega,Seafood Restaurant,Bar,Gym,Concert Hall
18,Downtown Toronto,4,Coffee Shop,Diner,Bank,Bar,Italian Restaurant,Sandwich Place,Japanese Restaurant,Distribution Center,Portuguese Restaurant,Hobby Shop
